# Finetune a ResNet image classifier

## Load data

In [ ]:
!curl -L https://storage.googleapis.com/aiolympiadmy/ioai-2025-tsp/finetuning-resnet.zip -o data.zip

In [ ]:
!unzip data.zip

## Establishing a baseline

Load an ImageNet pre-trained ResNet34, and check it's performance on the images in `data/test`.  

Use accuracy, precision and recall as your metrics for performance.

In [ ]:
# YOUR CODE BELOW

## Finetuning

Finetune this ResNet using data in `data/train`. Use `data/test` as your testing set, and use cross entropy loss. The rest is up to you. Run finetuning that terminates within approx 10 mins. Store the following info every 10 minibatches: loss, precision, recall and accuracy on train and test datasets.

## Writeup

Summarize what you did above, as well as detail the choices you made and why. What was the outcome?

## Further analysis

Pick one aspect about the work done above thus far that you find interesting, investigate it a bit further, and give a short paragraph writeup of what you investigated and how it went.